# Team member Details

- Aditya Mishra 17BCE0687
- Aditya Oke 18BCE0305

# Implementation Details

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
! pip install autoviz # We need this external package
! pip install pandas_profiling
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling as pp
from sklearn import preprocessing
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import urllib
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
# print(os.listdir('/kaggle/input/osmi-mental-health-in-tech-survey-2019/'))

In [0]:
BASE_DIR = '/content/'
OUTPUT_FILE = os.path.join(BASE_DIR, 'osmi_data.csv')

In [0]:
data_file = urllib.request.urlretrieve('https://raw.githubusercontent.com/oke-aditya/Mental_Health_Analysis/master/osmi_data.csv', OUTPUT_FILE)

In [0]:
# DATA_PATH = r'/kaggle/input/osmi-mental-health-in-tech-survey-2019/OSMI 2019 Mental Health in Tech Survey Results - OSMI Mental Health in Tech Survey 2019.csv'
# DATA_PATH = r'/content/OSMI 2019 Mental Health in Tech Survey Results - OSMI Mental Health in Tech Survey 2019.csv'

In [0]:
df = pd.read_csv(OUTPUT_FILE)

- There is lot of text data.
- We have many columns that are categorical and require cleaning.
- Also we have some subjective columns that have open-ended answers.
- It needs thorough cleaning before visualizing

In [0]:
df.head() # Let's see the data.

,*Are you self-employed?*,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",Would you feel more comfortable talking to your coworkers about your physical health or your mental health?,Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?,Have you ever discussed your mental health with your employer?,"Describe the conversation you had with your employer about your mental health, including their reactions and what actions were taken to address your mental health issue/questions.",Would you feel comfortable discussing a mental health issue with your coworkers?,Have you ever discussed your mental health with coworkers?,Describe the conversation with coworkers you had about your mental health including their reactions.,Have you ever had a coworker discuss their or another coworker's mental health with you?,Describe the conversation your coworker had with you about their mental health (please do not use names).,"Overall, how much importance does your employer place on physical health?","Overall, how much importance does your employer place on mental health?",Do you have medical coverage (private insurance or state-provided) that includes treatment of mental health disorders?,Do you know local or online resources to seek help for a mental health issue?,"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?","If you have revealed a mental health disorder to a client or business contact, how has this affected you or the relationship?","If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?","If you have revealed a mental health disorder to a coworker or employee, how has this impacted you or the relationship?",Do you believe your productivity is ever affected by a mental health issue?,"If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?",*Do you have previous employers?*,Was your employer primarily a tech company/organization?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health disorders and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Would you have felt more comfortable talking to your previous employer about your physical health or your mental health?,Would you have been willing to discuss your mental health with your direct supervisor(s)?,Did you ever discuss your mental health with your previous employer?,"Describe the conversation you had with your previous employer about your mental health, including their reactions and actions taken to address your mental health issue/questions.",...,Describe the conversation you had with your previous coworker

# Data cleaning Functions.

- We wrote these functions that can be used by anyone to clean the data.
- They are useful to use multiple times.

In [0]:
def check_missing(column, check_df):
    if(check_df[column].isnull().any()):
        print("Number of rows having the NULL value in ",check_df," are ",check_df[column].isnull().sum())
    else:
        print("There is no NULL vale in the column", column)

In [0]:
def check_missing_all(check_df, verbose=False):
    empty_list = []
    for column in df.columns:
        if(check_df[column].isnull().any()):
            empty_rows = check_df[column].isnull().sum()
            empty_list.append(empty_rows)
            
            if(verbose == True):
                print("Number of rows having the NULL value in column %s are %d " %(column, empty_rows))
        else:
            print("There is no NULL value in the column %s" %(column))
            empty_list.append(0)
            
    return empty_list

In [0]:
def convert_columns(df):
    ''' Given a datarame numbers the columns'''
    column_names = {}
    start_i = 1
    i = 1
    for column_name in df.columns:
        column_names[column_name] = str(i)
        i += 1
    
    df.rename(column_names, axis=1, inplace=True)
    print("Columns renamed strating with i = %d to i = %d" %(start_i,i))
    return df, column_names
#         print(column_name)
    

In [0]:
def clean_missing(df, column_list, empty_rows, percent_del=0.65):
    ''' Given a percentage it will remove columns that have data missing below that percentage.'''
    count = 0
    dropped_columns = []
    for i, column in enumerate(column_list):
        calc_percent = empty_rows[i] / len(df)
        if(calc_percent > percent_del):
            df.drop([column], axis=1, inplace=True)
            dropped_columns.append(column)
            print("%s Column dropped "%(column))
            count += 1
    
    print("Number of columns dropped %d"%(count))

    return df, dropped_columns

In [0]:
def encode_categorical(df, column_list):
    ''' Encodes categorical column using Ordinal Encoder '''
    for column in column_list:
        df[column] = df[column].astype('str')
        encoder = preprocessing.LabelEncoder()
        encoded_list = encoder.fit_transform(df[column])
#         print(encoded_list)
#         print(len(encoded_list))
        encoded_series = pd.Series(encoded_list)
        df[column] = encoded_series
        print("The ", column, "is encoded ")
    return(df)

In [0]:
# Numeric types need to do MinxMaxScaler
def scale_data(df, column_list, index_list):
    ''' Standardizes the data '''
    for column in column_list:
        df[column] = df[column].astype('float')
        encoder = preprocessing.StandardScaler()
        df[column] = encoder.fit_transform(df[column].values.reshape(-1,1))
        print("The ",column, "is encoded")
    return(df)

# Data Cleaning I

In [0]:
empty_rows = check_missing_all(df, verbose=False)

There is no NULL value in the column *Are you self-employed?*
There is no NULL value in the column *Do you have previous employers?*
There is no NULL value in the column Do you *currently* have a mental health disorder?
There is no NULL value in the column Have you ever sought treatment for a mental health disorder from a mental health professional?
There is no NULL value in the column Do you have a family history of mental illness?
There is no NULL value in the column If you have a mental health disorder, how often do you feel that it interferes with your work *when being treated effectively?*
There is no NULL value in the column If you have a mental health disorder, how often do you feel that it interferes with your work *when* _*NOT*_* being treated effectively (i.e., when you are experiencing symptoms)?*
There is no NULL value in the column How willing would you be to share with friends and family that you have a mental illness?
There is no NULL value in the column Would you be wil

In [0]:
print("Number of rows in our dataset are %d "%(len(df)))
print("Number of columns in our dataset are %d "%(df.shape[1]))
# print(len(empty_rows))
print("Column wise count of empty rows: \n ",empty_rows)

Number of rows in our dataset are 352 
Number of columns in our dataset are 82 
Column wise count of empty rows: 
  [0, 48, 48, 48, 48, 74, 48, 48, 48, 48, 48, 48, 48, 268, 48, 51, 233, 48, 242, 48, 48, 304, 304, 304, 332, 304, 304, 304, 315, 0, 56, 56, 56, 56, 56, 56, 56, 56, 56, 289, 56, 58, 267, 57, 286, 56, 56, 0, 205, 352, 280, 208, 3, 0, 0, 0, 0, 70, 0, 0, 79, 0, 67, 0, 307, 334, 0, 0, 263, 0, 352, 0, 98, 252, 0, 0, 5, 0, 148, 148, 0, 148]


In [0]:
# We drop columns that have lot of missing data.
# Probably people do not like to answer these questions.
df, dropped_columns = clean_missing(df, list(df.columns), empty_rows, percent_del=0.65)

Describe the conversation you had with your employer about your mental health, including their reactions and what actions were taken to address your mental health issue/questions. Column dropped 
Describe the conversation with coworkers you had about your mental health including their reactions. Column dropped 
Describe the conversation your coworker had with you about their mental health (please do not use names). Column dropped 
Do you have medical coverage (private insurance or state-provided) that includes treatment of mental health disorders? Column dropped 
Do you know local or online resources to seek help for a mental health issue? Column dropped 
If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts? Column dropped 
If you have revealed a mental health disorder to a client or business contact, how has this affected you or the relationship? Column dropped 
If you have been diagnosed or treated for a mental he

In [0]:
# again let us see what is left in dataset.
empty_rows = check_missing_all(df, verbose=True)

There is no NULL value in the column *Are you self-employed?*
Number of rows having the NULL value in column How many employees does your company or organization have? are 48 
Number of rows having the NULL value in column Is your employer primarily a tech company/organization? are 48 
Number of rows having the NULL value in column Is your primary role within your company related to tech/IT? are 48 
Number of rows having the NULL value in column Does your employer provide mental health benefits as part of healthcare coverage? are 48 
Number of rows having the NULL value in column Do you know the options for mental health care available under your employer-provided health coverage? are 74 
Number of rows having the NULL value in column Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)? are 48 
Number of rows having the NULL value in column Does your employer offer resources to learn more about mental hea

In [0]:
print("Number of rows in our dataset are %d "%(len(df)))
print("Number of columns in our dataset are %d "%(df.shape[1]))
print(len(empty_rows))
print("Column wise count of empty rows: \n ",empty_rows)

Number of rows in our dataset are 352 
Number of columns in our dataset are 61 
61
Column wise count of empty rows: 
  [0, 48, 48, 48, 48, 74, 48, 48, 48, 48, 48, 48, 48, 48, 51, 48, 48, 48, 0, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 58, 57, 56, 56, 0, 205, 208, 3, 0, 0, 0, 0, 70, 0, 0, 79, 0, 67, 0, 0, 0, 0, 0, 98, 0, 0, 5, 0, 148, 148, 0, 148]


In [0]:
# We need to convert the text columns to numbered columns. 
# It will be hard to do column operations with text columns
# df, names = convert_columns(df)

In [0]:
# These are names of our new columns
# print(list(df.columns))

# After the preliminary data cleaning

This is what our new deta looks as of now.

We might need to renmae the columns. It won't be easy to access otherwise.


In [0]:
df.head()

,*Are you self-employed?*,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",Would you feel more comfortable talking to your coworkers about your physical health or your mental health?,Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?,Have you ever discussed your mental health with your employer?,Would you feel comfortable discussing a mental health issue with your coworkers?,Have you ever discussed your mental health with coworkers?,Have you ever had a coworker discuss their or another coworker's mental health with you?,"Overall, how much importance does your employer place on physical health?","Overall, how much importance does your employer place on mental health?",*Do you have previous employers?*,Was your employer primarily a tech company/organization?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health disorders and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Would you have felt more comfortable talking to your previous employer about your physical health or your mental health?,Would you have been willing to discuss your mental health with your direct supervisor(s)?,Did you ever discuss your mental health with your previous employer?,Would you have been willing to discuss your mental health with your coworkers at previous employers?,Did you ever discuss your mental health with a previous coworker(s)?,Did you ever have a previous coworker discuss their or another coworker's mental health with you?,"Overall, how much importance did your previous employer place on physical health?","Overall, how much importance did your previous employer place on mental health?",Do you *currently* have a mental health disorder?,Have you ever been *diagnosed* with a mental health disorder?,"*If so, what disorder(s) were you diagnosed with?*",Have you had a mental health disorder in the past?,Have you ever sought treatment for a mental health disorder from a mental health professional?,Do you have a family history of mental illness?,"If you have a mental health disorder, how often do you feel that it interferes with your work *when being treated effectively?*","If you have a mental health disorder, how often do you feel that it interferes with your work *when* _*NOT*_* being treated effectively (i.e., when you are experiencing symptoms)?*",Have your observations of how another individual who discussed a mental health issue made you less likely to reveal a mental health issue yourself in your current workplace?,How willing would you be to share with friends and family that you have a mental illness?,Would you be willing to bring up a physical health issue with a potential employer in an interview?,Why or why not?,Would you bring up your *mental* health with a potentia

In [0]:
# print(names)
# These are the set of questoins that were asked.
print(df.columns)

In [0]:
# Let us save this csv as we will use later.
df.to_csv('cleaned_osmi.csv', index=False)

# Data exploration and EDA

- We opted for Pandas profiling to get exploration of data.
- Vizualization is not effective.
- We used auto data viz that will help us visualize.
- It would be very lengthy to get EDA done also that was not our motive.

In [0]:
from autoviz.AutoViz_Class import AutoViz_Class

In [0]:
av = AutoViz_Class()

In [0]:
# The output of autoviz is not satisfactory. We will look into it. This maybe because columns are many.
# We have commented the below cell but you may run it.

In [0]:
# dft = av.AutoViz('/content/cleaned_osmi.csv', sep=',', depVar='', header=1, verbose=1, lowess=False, chart_format='svg',
# max_rows_analyzed=7500, max_cols_analyzed=80)

In [0]:
pp.ProfileReport(df)
# The output is huge. Please take time to view it.
# Doing this EDA is difficult

Number of variables,61
Number of observations,352
Total Missing (%),13.1%
Total size in memory,155.8 KiB
Average record size in memory,453.4 B
Numeric,8
Categorical,39
Boolean,5
Date,0
Text (Unique),0
Rejected,0


# Data Cleaning II

- Here we convert the categorical variables to dummy variables.
- We rename the columns so that it becomes easy to access

In [0]:
# Renmaing the columns
df, names = convert_columns(df)

Columns renamed strating with i = 1 to i = 62


In [0]:
print(names)

{'*Are you self-employed?*': '1', 'How many employees does your company or organization have?': '2', 'Is your employer primarily a tech company/organization?': '3', 'Is your primary role within your company related to tech/IT?': '4', 'Does your employer provide mental health benefits as part of healthcare coverage?': '5', 'Do you know the options for mental health care available under your employer-provided health coverage?': '6', 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?': '7', 'Does your employer offer resources to learn more about mental health disorders and options for seeking help?': '8', 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?': '9', 'If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?': '10', 'Would you f

In [0]:
df.head(3)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61
0,False,26-100,True,True,I don't know,No,Yes,Yes,I don't know,Very easy,Physical health,Yes,False,Yes,True,True,5.0,5.0,True,False,I don't know,N/A (was not aware),Some did,"Yes, they all did",I don't know,Physical health,Some of my previous supervisors,False,At some of my previous employers,False,True,8.0,5.0,Don't Know,NaN,NaN,No,False,No,Not applicable to me,Not applicable to me,No,8,Yes,If it impacts work I would bring it up,Maybe,If I feel it's under control I likely wouldn't...,False,8,No,Maybe/Not sure,4,NaN,False,25,Male,United States of America,Nebraska,White,United States of America,Nebraska
1,False,26-100,True,True,Yes,No,No,Yes,Yes,I don't know,Physical health,Maybe,False,Maybe,True,True,5.0,7.0,True,False,"Yes, they all did",I was aware of some,Some did,Some did,"Yes, always",Physical health,Some of my previous supervisors,False,At some of my previous employers,True,True,8.0,5.0,Possibly,NaN,NaN,Possibly,False,Yes,Sometimes,Often,No,7,No,I do not trust that the potential employer wou...,No,I do not trust that the potential employer wou...,False,7,"Yes, I observed","Yes, I experienced",1,NaN,False,51,male,United States of America,Nebraska,White,United States of America,Nebraska
2,False,26-100,True,True,I don't know,No,No,I don't know,I don't know,Somewhat difficult,Same level of comfort for each,No,False,Maybe,True,True,5.0,4.0,True,True,I don't know,N/A (was not aware),None did,None did,I don't know,Physical health,"No, none of my previous supervisors",False,"No, at none of my previous employers",False,False,10.0,2.0,No,NaN,NaN,No,False,I don't know,Not applicable to me,Not applicable to me,No,4,Maybe,It depends on its applicability to my job and ...,No,The stigma attached to mental health and not k...,False,4,Maybe/Not sure,"Yes, I observed",2,I think opening up more conversation around th...,True,27,Male,United States of America,Illinois,White,United States of America,Illinois


In [0]:
df.tail(3)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61
349,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,False,No,Never,Never,NaN,10,Yes,dhdhdthhusshshshdhkhsdhsdljsdkhsdskh,No,dksdhdkjhwkdhkhwkwhkwhwke,False,10,I've always been self-employed,I've always been self-employed,5,sdsdsdsdsdsd,False,50,M,India,NaN,NaN,India,NaN
350,False,More than 1000,True,True,No,No,No,No,Yes,Difficult,Physical health,No,False,Maybe,False,False,9.0,3.0,True,True,"No, none did",N/A (was not aware),None did,None did,I don't know,Same level of comfort for each,I don't know,False,"No, at none of my previous employers",False,False,8.0,2.0,Possibly,NaN,NaN,No,False,No,Often,Often,Yes,2,No,not comfortable,No,not comfortable,False,2,Maybe/Not sure,"Yes, I experienced",2,raise awareness,False,30,female,India,NaN,NaN,India,NaN
351,False,More than 1000,True,True,Yes,Yes,No,Yes,Yes,Somewhat difficult,Physical health,Maybe,True,No,True,False,5.0,3.0,True,False,"No, none did",N/A (none offered),Some did,Some did,I don't know,Physical health,Some of my previous supervisors,False,At some of my previous employers,True,False,8.0,3.0,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,True,Yes,Sometimes,Often,NaN,10,Maybe,my physical health issues do not affect my abi...,Yes,I would bring it up so that I could negotiate ...,True,3,Maybe/Not sure,No,3,reduce stigma. offer options for part time wor...,True,24,Female (cis),United States of America,Oregon,White,United States of America,Oregon


In [0]:
# Now we will convert the categorical columns. We wrote the conversion function on top.

In [0]:
categorical_columns = ['1', '3','4','5','6','7','8', '9' ,'10'] # just showing a sample of columns. We will extend this to all categorical columns.

In [0]:
df = encode_categorical(df, categorical_columns)

The  1 is encoded 
The  3 is encoded 
The  4 is encoded 
The  5 is encoded 
The  6 is encoded 
The  7 is encoded 
The  8 is encoded 
The  9 is encoded 
The  10 is encoded 


In [0]:
df.head() # We see that we replaced the categorical variables with numbers

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61
0,0,26-100,1,1,0,0,2,2,0,5,Physical health,Yes,False,Yes,True,True,5.0,5.0,True,False,I don't know,N/A (was not aware),Some did,"Yes, they all did",I don't know,Physical health,Some of my previous supervisors,False,At some of my previous employers,False,True,8.0,5.0,Don't Know,NaN,NaN,No,False,No,Not applicable to me,Not applicable to me,No,8,Yes,If it impacts work I would bring it up,Maybe,If I feel it's under control I likely wouldn't...,False,8,No,Maybe/Not sure,4,NaN,False,25,Male,United States of America,Nebraska,White,United States of America,Nebraska
1,0,26-100,1,1,3,0,1,2,2,1,Physical health,Maybe,False,Maybe,True,True,5.0,7.0,True,False,"Yes, they all did",I was aware of some,Some did,Some did,"Yes, always",Physical health,Some of my previous supervisors,False,At some of my previous employers,True,True,8.0,5.0,Possibly,NaN,NaN,Possibly,False,Yes,Sometimes,Often,No,7,No,I do not trust that the potential employer wou...,No,I do not trust that the potential employer wou...,False,7,"Yes, I observed","Yes, I experienced",1,NaN,False,51,male,United States of America,Nebraska,White,United States of America,Nebraska
2,0,26-100,1,1,0,0,1,0,0,3,Same level of comfort for each,No,False,Maybe,True,True,5.0,4.0,True,True,I don't know,N/A (was not aware),None did,None did,I don't know,Physical health,"No, none of my previous supervisors",False,"No, at none of my previous employers",False,False,10.0,2.0,No,NaN,NaN,No,False,I don't know,Not applicable to me,Not applicable to me,No,4,Maybe,It depends on its applicability to my job and ...,No,The stigma attached to mental health and not k...,False,4,Maybe/Not sure,"Yes, I observed",2,I think opening up more conversation around th...,True,27,Male,United States of America,Illinois,White,United States of America,Illinois
3,0,100-500,1,1,0,0,2,2,2,5,Physical health,Yes,False,Maybe,False,True,1.0,3.0,True,True,I don't know,N/A (was not aware),Some did,Some did,"Yes, always",Physical health,Some of my previous supervisors,False,At some of my previous employers,False,True,3.0,1.0,No,NaN,NaN,No,False,Yes,Not applicable to me,Not applicable to me,No,3,No,discrimination/bias,No,discrimination/bias,False,8,No,"Yes, I observed",3,NaN,False,37,male,United States of America,Nebraska,White,United States of America,Nebraska
4,0,26-100,1,1,0,0,0,0,0,1,Physical health,No,False,No,False,False,4.0,4.0,True,True,I don't know,N/A (was not aware),None did,None did,I don't know,Physical health,"No, none of my previous supervisors",False,"No, at none of my previous employers",False,False,4.0,4.0,No,NaN,NaN,No,False,No,Not applicable to me,Not applicable to me,NaN,1,No,NaN,No,NaN,False,5,No,No,3,NaN,False,46,m,United States of America,Nebraska,White,United States of America,Nebraska


In [0]:
df.tail()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61
347,0,More than 1000,0,1,0,0,1,0,0,3,Physical health,No,False,No,False,False,5.0,5.0,True,False,"No, none did",N/A (none offered),I don't know,None did,I don't know,Physical health,"No, none of my previous supervisors",False,"No, at none of my previous employers",False,False,5.0,1.0,Possibly,NaN,NaN,No,False,No,Sometimes,Sometimes,Maybe,9,Maybe,NaN,No,NaN,False,1,"Yes, I observed",Maybe/Not sure,3,NaN,False,27,male,India,NaN,NaN,India,NaN
348,1,NaN,2,2,4,2,3,3,3,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,"No, none did",N/A (none offered),None did,None did,I don't know,Physical health,"No, none of my previous supervisors",False,"No, at none of my previous employers",False,False,5.0,0.0,No,NaN,NaN,No,False,No,Not applicable to me,Not applicable to me,NaN,3,Maybe,would depend on issue and to what degree it wo...,Maybe,"same as with the physical issue, but here depe...",False,2,"Yes, I observed",No,3,IDK,False,48,m,United States of America,Louisiana,White,United States of America,Louisiana
349,1,NaN,2,2,4,2,3,3,3,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,False,No,Never,Never,NaN,10,Yes,dhdhdthhusshshshdhkhsdhsdljsdkhsdskh,No,dksdhdkjhwkdhkhwkwhkwhwke,False,10,I've always been self-employed,I've always been self-employed,5,sdsdsdsdsdsd,False,50,M,India,NaN,NaN,India,NaN
350,0,More than 1000,1,1,1,0,1,1,2,0,Physical health,No,False,Maybe,False,False,9.0,3.0,True,True,"No, none did",N/A (was not aware),None did,None did,I don't know,Same level of comfort for each,I don't know,False,"No, at none of my previous employers",False,False,8.0,2.0,Possibly,NaN,NaN,No,False,No,Often,Often,Yes,2,No,not comfortable,No,not comfortable,False,2,Maybe/Not sure,"Yes, I experienced",2,raise awareness,False,30,female,India,NaN,NaN,India,NaN
351,0,More than 1000,1,1,3,1,1,2,2,3,Physical health,Maybe,True,No,True,False,5.0,3.0,True,False,"No, none did",N/A (none offered),Some did,Some did,I don't know,Physical health,Some of my previous supervisors,False,At some of my previous employers,True,False,8.0,3.0,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,True,Yes,Sometimes,Often,NaN,10,Maybe,my physical health issues do not affect my abi...,Yes,I would bring it up so that I could negotiate ...,True,3,Maybe/Not sure,No,3,reduce stigma. offer options for part time wor...,True,24,Female (cis),United States of America,Oregon,White,United States of America,Oregon


# Difficulties Faced during Implementation

- Many columns with language as well. 
- Data cleaning and feature engineering takes time and lot of thinking.
- We used automated data exploration as it would take tremendous code and effort to generate effective visualization.
- It was not easy to understand the correlation and use domain knowledge to find causalities.
- Machine Learning aspect was not very hard. Tougher was data wrangling.

# Percentage of Work completed so far.

- We have done Exploratory Data analysis (EDA), visualization.
- Also cleaned the data to certain extent.
- We need to call our functoins again and again and get the job done.
- Functions are ready !!! So nothing much to worry.
- We need to model the data as per our needs.
- In our view almost 50 % implementation is done.